# Домашнее задание №2 

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [1]:
import datetime
import re

PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}

PERIODICITY = {
    'weekly': 7,
    'monthly': 30,
    'yearly': 365
}

class ReadableEntity:
    def __init__(self, title, author, genre, page_num, page_format, rarity_index, text):
        self.title = title
        self.author = author
        self.genre = genre    
        self.text = text
        if type(page_num) == int and page_num > 0:
            self.page_num = page_num
        else:
            raise ValueError("Number of pages should be positive integer")
        if page_format in PAGES_FORMAT.keys():
            self.page_format = page_format
        else:
            raise ValueError("Non-standardized page format. Expected one of this: {}".format(list(PAGES_FORMAT.keys())))
        if type(rarity_index) == int and rarity_index >=1 and rarity_index <= 10:
            self.rarity_index = rarity_index
        else:
            raise ValueError("Rarity index should be an integer from interval [1, 10]")
    
    def __repr__(self):
        return '{} "{}" ({}). R.I.{} - {} p.({})'.format(self.author, self.title, self.genre,
                                                             self.rarity_index, self.page_num, self.page_format)
    
    @property
    def pages_square(self):
        height, width = PAGES_FORMAT[self.page_format]
        return height*width*self.page_num
        

class Journal(ReadableEntity):
    def __init__(self, title, author, genre, page_num, page_format, rarity_index, text, issue_date, first_issue_date,
                 periodicity):
        super().__init__(title, author, genre, page_num, page_format, rarity_index, text)
        try:
            self.issue_date = datetime.date(*[int(i) for i in issue_date.split('/')])
        except Exception:
            raise Exception("Issue date should string wuth format YYYY/MM/DD, got '{}'".format(issue_date))
        try:
            self.first_issue_date = datetime.date(*[int(i) for i in first_issue_date.split('/')])
        except Exception:
            raise Exception("First issue date should string wuth format YYYY/MM/DD, got '{}'".format(first_issue_date))
        if self.first_issue_date > self.issue_date:
            raise ValueError("First issue date should be less then issue date")
        if periodicity in PERIODICITY.keys():
            self.periodicity = periodicity
        else:
            raise ValueError("Non-standardized periodicity. Expected one of this: {}".format(list(PERIODICITY.keys())))
    
    def __repr__(self):
        return '{} "{}" ({}) - {}. Vol.{} No.{}. R.I.{} - {} p.({})'.format(
            self.author, self.title, self.genre, self.issue_date.year, self.issue_volume, self.issue_num,
            self.rarity_index, self.page_num, self.page_format
        )
    
    @property
    def issue_num(self):
        if self.issue_date == self.first_issue_date.year:
            first_date = self.first_issue_date
        else:
            first_date = datetime.date(self.issue_date.year, 1, 1)
        delta = (self.issue_date - first_date).days
        return delta // PERIODICITY[self.periodicity] + 1
    
    @property
    def issue_volume(self):
        first_date = datetime.date(self.first_issue_date.year+1, 1, 1)
        delta = (self.issue_date - first_date).days
        return delta // 365 + 2


class Book(ReadableEntity):
    def __init__(self, title, author, genre, page_num, page_format, rarity_index, text, edition_num, publication_year):
        super().__init__(title, author, genre, page_num, page_format, rarity_index, text)
        if type(edition_num) == int and edition_num > 0:
            self.edition_num = edition_num
        else:
            raise ValueError("Edition number should be positive integer")
        if type(publication_year) == int and publication_year > 0:
            self.publication_year = publication_year
        else:
            raise ValueError("Publication year should be positive integer")
    
    def __repr__(self):
        return '{} "{}" ({}). - {} edition, {}. R.I.{} - {} p.({})'.format(self.author, self.title, self.genre,
                self.edition_num, self.publication_year, self.rarity_index, self.page_num, self.page_format)

class Exporter:
    def __init__(self, title, rarity_index):
        if rarity_index in [9, 10]:
            storage_path = re.sub(' ', '_', title.lower())
            storage_path = re.sub('\W_', '', storage_path)
            self.export_to_storage(storage_path[:max(20, len(storage_path))]+'.json')
            
    def export_to_storage(self, storage_path):
        with open(storage_path, 'w') as storage:
            for key in self.__dict__:
                storage.write("{}: {}\n".format(key, self.__dict__[key]))
    
class LibraryJournal(Journal, Exporter):
    def __init__(self, title, author, genre, page_num, page_format, rarity_index, text, issue_date, first_issue_date,
                 periodicity):
        Journal.__init__(self, title, author, genre, page_num, page_format, rarity_index, text, issue_date, first_issue_date,
                 periodicity)
        Exporter.__init__(self, title, rarity_index)
        self.is_taken = False
        
    def take_from_library(self):
        if self.is_taken:
            raise ValueError("Exemplar of journal {} No.{} is aready taken from library".format(self.title, self.issue_num))
        else:
            self.is_taken = True
        
    def return_to_library(self):
        if self.is_taken:
            self.is_taken = False
            print("You've taken exemplar of journal {} No.{}".format(self.title, self.issue_num))
        else:
            raise ValueError("This exemplar of journal {} No.{} is not from the library".format(self.title, self.issue_num))

class LibraryBook(Book, Exporter):
    def __init__(self, title, author, genre, page_num, page_format, rarity_index, text, edition_num, publication_year,
                 num_copies):
        Book.__init__(self, title, author, genre, page_num, page_format, rarity_index, text, edition_num, publication_year)
        Exporter.__init__(self, title, rarity_index)
        if type(num_copies) == int and num_copies > 0:
            self.current_num_copies = num_copies
            self.total_num_copies = num_copies
        else:
            raise ValueError("Number of pages should be positive integer")
        
    def take_from_library(self):
        if self.current_num_copies > 0:
            self.current_num_copies -= 1
            print("You've taken exemplar of {}".format(self.title))
        else:
            raise ValueError("All exemplar of {} are already taken from library".format(self.title))
    
    def return_to_library(self):
        if self.current_num_copies < self.total_num_copies:
            self.current_num_copies += 1
        else:
            raise ValueError("This exemplar of {} is not from the library".format(self.title))

In [2]:
lb1 = LibraryBook(
    "Introduction to algorithms","T.Cormen","science", 1312, "A4", 6, edition_num=3, publication_year=2011, num_copies=2,
    text="Chapter 1. The Role of Algorithms in Computing. \nWhat are algorithms? Why is the study of algorithms worthwhile? What is the role of algorithms relative to other technologies used in computers? In this chapter, we will answer these questions."
)
lb1

T.Cormen "Introduction to algorithms" (science). - 3 edition, 2011. R.I.6 - 1312 p.(A4)

In [3]:
lb2 = LibraryBook(
    "The Invisible Man","H.G.Wells", "novel", 288, "A4", 9,  edition_num=1, publication_year=1897, num_copies=1,
    text="Chapter I. The Strange Man's arrival. \nThe stranger came early in February, one wintry day, through a biting wind and a driving snow, the last snowfall of the year, over the down, walking as it seemed from Bramblehurst railway station, and carrying a little black portmanteau in his thickly gloved hand.",
)
lb2

H.G.Wells "The Invisible Man" (novel). - 1 edition, 1897. R.I.9 - 288 p.(A4)

In [5]:
lj1 = LibraryJournal("International Journal of Business and Social Science","Frank Tian Xie","business", 37, "A3", 4,
                     issue_date="2016/05/01", first_issue_date="2010/10/01", periodicity="monthly",
                     text="Article 1\nArticle 2\nArticle 3\nArticle 4")
lj1

Frank Tian Xie "International Journal of Business and Social Science" (business) - 2016. Vol.7 No.5. R.I.4 - 37 p.(A3)

In [6]:
lb2.take_from_library()
lb2.take_from_library()

You've taken exemplar of The Invisible Man


ValueError: All exemplar of The Invisible Man are already taken from library

In [7]:
lb2.return_to_library()
lb2.take_from_library()

You've taken exemplar of The Invisible Man


In [8]:
lb2.pages_square

17962560

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [9]:
import sys

def get_real_sizeof(obj, checked_objects=None):
    if checked_objects is None:
        checked_objects = []
    size = 0
    if id(obj) not in checked_objects:
        checked_objects.append(id(obj))
        if type(obj) == dict:
            size += sys.getsizeof(dict())
            for key, value in obj.items():
                size += get_real_sizeof(key, checked_objects)
                size += get_real_sizeof(value, checked_objects)
        elif hasattr(obj, '__iter__'):
            if type(obj) in [list, tuple, str]:
                size += sys.getsizeof(type(obj)())
            else:
                size += sys.getsizeof(obj)
            for subobj in obj.__iter__():
                size += get_real_sizeof(subobj, checked_objects)
        elif hasattr(obj, '__dict__'):
            size += get_real_sizeof(obj.__dict__, checked_objects)
        else:
            size += sys.getsizeof(obj)
    return size

In [12]:
for obj in ['aaaaaaaaaaa', ['aaaaaaaaaaa'], bytearray(12), complex(1, 3), 670, -1111111111111111,
            (1,2,3,4), {'a': 0, 'b': 1, 'c': -1}, Polynom([1,2,3,4,5]), int]:
    print('Object: {} (type={})\nsys.getsizeof: {}, real size: {}\n'.format(obj, type(obj),
                                                                       sys.getsizeof(obj), get_real_sizeof(obj)))

Object: aaaaaaaaaaa (type=<class 'str'>)
sys.getsizeof: 60, real size: 98

Object: ['aaaaaaaaaaa'] (type=<class 'list'>)
sys.getsizeof: 72, real size: 162

Object: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00') (type=<class 'bytearray'>)
sys.getsizeof: 69, real size: 93

Object: (1+3j) (type=<class 'complex'>)
sys.getsizeof: 32, real size: 32

Object: 670 (type=<class 'int'>)
sys.getsizeof: 28, real size: 28

Object: -1111111111111111 (type=<class 'int'>)
sys.getsizeof: 32, real size: 32

Object: (1, 2, 3, 4) (type=<class 'tuple'>)
sys.getsizeof: 80, real size: 160

Object: {'a': 0, 'b': 1, 'c': -1} (type=<class 'dict'>)
sys.getsizeof: 240, real size: 467

Object: 5*x^4 + 4*x^3 + 3*x^2 + 2*x + 1 (type=<class '__main__.Polynom'>)
sys.getsizeof: 56, real size: 885

Object: <class 'int'> (type=<class 'type'>)
sys.getsizeof: 400, real size: 4311



### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [13]:
from copy import copy

SIGN = {True: "+", False: "-"}

class Polynom:
    
    def __init__(self, coefficients):
        self.coefficients = self.__convert_coefficients(coefficients)
    
    @staticmethod
    def __convert_coefficients(coefficients):
        if type(coefficients) not in [list, tuple, int, float, complex]:
            raise TypeError("Coefficients should be of type list, tuple, int, float or complex")
        if type(coefficients) in [int, float, complex]:
            coefficients = [coefficients]    
        return coefficients  
    
    def __check_degrees(self, other):
        self_coefficients = copy(self.coefficients)
        if type(other) == type(self):
            other_coefficients = copy(other.coefficients)
        else:
            other_coefficients = self.__convert_coefficients(other)
        
        swap_flag = False
        if len(self_coefficients) < len(other_coefficients):
            swap_flag = True
            self_coefficients, other_coefficients = other_coefficients, self_coefficients
        return self_coefficients, other_coefficients, swap_flag
    
    # 1. сложение
    def __add__(self, other):
        result_coefficients, other_coefficients, _ = self.__check_degrees(other)
        for degree in range(len(other_coefficients)):
            result_coefficients[degree] += other_coefficients[degree]
        
        return Polynom(result_coefficients)
    
    # 2. вычитание
    def __sub__(self, other):
        result_coefficients, other_coefficients, swap_flag = self.__check_degrees(other)
        if swap_flag:
            for degree in range(len(other_coefficients)):
                    result_coefficients[degree] = other_coefficients[degree] - result_coefficients[degree]
            for last_degree in range(degree+1, len(result_coefficients)):
                result_coefficients[last_degree] = -result_coefficients[last_degree]
        else:
            for degree in range(len(other_coefficients)):
                result_coefficients[degree] -= other_coefficients[degree]
                
        return Polynom(result_coefficients)
    
    # 3. умножение
    def __mul__(self, other):
        self_coefficients, other_coefficients, _ = self.__check_degrees(other)
        result_coefficients = [0]*(len(self_coefficients)+len(other_coefficients))
        for degree_i, coef_i in enumerate(self_coefficients):
            for degree_j, coef_j in enumerate(other_coefficients):
                result_coefficients[degree_i+degree_j] += (coef_i*coef_j)
        return Polynom(result_coefficients)
    
    # 3a. умножение Карацубы
    def karatsuba_multiply(self, other):
        pass
    
    # 4. деление
    def __truediv__(self, other):
        self_coefficients, divider_coefficients, swap_flag = self.__check_degrees(other)
        if swap_flag:
            quotient_coefficients = 0
            remainder_coefficients = divider_coefficients
        else:
            quotient_coefficients = []
            dividend_coefficients = self_coefficients[-len(divider_coefficients)+1:]
            for step in range(len(self_coefficients)-len(divider_coefficients)+1):
                remainder_coefficients = []
                dividend_coefficients.insert(0, self_coefficients[-len(divider_coefficients)-step])
                coef = dividend_coefficients[-1] / divider_coefficients[-1]
                quotient_coefficients.insert(0, coef)
                for degree in range(len(divider_coefficients)-1):
                    remainder_coefficients.append(dividend_coefficients[degree] - divider_coefficients[degree]*coef)
                dividend_coefficients = remainder_coefficients
        return Polynom(quotient_coefficients), Polynom(remainder_coefficients)               

    # 5. быстрое возведение в степень
    def __pow__(self, n):
        powers = bin(n)[2:]
        result = copy(self)
        for i in range(1, len(powers)):
            if powers[i] == '0':
                result = result*result
            else:
                result = self*(result*result)
        return result
    
    # 6. представление многочлена в человеческом виде
    def __repr__(self):
        polynom_str = ""
        for degree, coef in enumerate(self.coefficients):
            element_str = ""
            if degree != 0:
                element_str = "x"
                if degree != 1:
                    element_str += "^{}".format(degree)
                if type(coef) == complex:
                    element_str = " + {}*".format(coef) + element_str
                elif coef == 0:
                    if len(self.coefficients) == 1:
                        element_str = " + 0"
                    else:
                        element_str = ""
                elif abs(coef) != 1:
                    element_str = " {} {}*".format(SIGN[coef > 0], round(abs(coef),3)) + element_str
                elif abs(coef) == 1:
                    element_str = " {} ".format(SIGN[coef > 0]) + element_str
            else:
                if type(coef) == complex:
                    element_str = " + {}".format(coef)
                elif coef == 0 and len(self.coefficients) == 1: 
                    element_str = " + 0"
                elif coef != 0:
                    element_str = " {} {}".format(SIGN[coef > 0], round(abs(coef),3)) + element_str
            polynom_str = element_str + polynom_str 
        return polynom_str.lstrip('+ ').rstrip('*')
    
    # 7. дифференцирование
    def differentiate(self):
        result_coefficients = []
        for degree in range(1, len(self.coefficients)):
            result_coefficients.append(self.coefficients[degree]*degree)
        return Polynom(result_coefficients)
    
    # 8. интегрирование
    def integrate(self, const=0):
        result_coefficients = [const]
        for degree in range(len(self.coefficients)):
            result_coefficients.append(self.coefficients[degree]/(degree+1))
        return Polynom(result_coefficients)
    
    # 9. вычисление значения в точке
    def __call__(self, x):
        result = 0
        for degree, coef in enumerate(self.coefficients):
            result += (coef*(x**degree))
        return result


In [14]:
print('Real:')
a = Polynom([1,8,-2,3,0,1])
print('a =', a)

b = Polynom([-2,1,1])
print('b =', b)

c = Polynom([0, 2, 1, -5])
print('c =', c)

print('\nComplex:')
d = Polynom([1+3j, -2, 3-1j, 4j, 5])
print('d =', d)

e = Polynom([-4-5j, 2])
print('e =', e)

Real:
a = x^5 + 3*x^3 - 2*x^2 + 8*x + 1
b = x^2 + x - 2
c = - 5*x^3 + x^2 + 2*x

Complex:
d = 5*x^4 + 4j*x^3 + (3-1j)*x^2 - 2*x + (1+3j)
e = 2*x + (-4-5j)


In [15]:
# 1. сложение
print('Real:')
print('a + b =', a + b)
print('b + 4 =', b + 4)
print('\nComplex:')
print('c + d =', c + d)
print('d + 9 =', d + 9)

Real:
a + b = x^5 + 3*x^3 - x^2 + 9*x - 1
b + 4 = x^2 + x + 2

Complex:
c + d = 5*x^4 + (-5+4j)*x^3 + (4-1j)*x^2 + (1+3j)
d + 9 = 5*x^4 + 4j*x^3 + (3-1j)*x^2 - 2*x + (10+3j)


In [16]:
# 2. вычитание
print('Real:')
print('a - b =', a - b)
print('b - c =', b - c)
print('\nComplex:')
print('d - e =', d - e)

Real:
a - b = x^5 + 3*x^3 - 3*x^2 + 7*x + 3
b - c = 5*x^3 - x - 2

Complex:
d - e = 5*x^4 + 4j*x^3 + (3-1j)*x^2 - 4*x + (5+8j)


In [17]:
# 3. умножение
print('Real:')
print('a * b =', a * b)
print('\nComplex:')
print('d * e =', d * e)

Real:
a * b = x^7 + x^6 + x^5 + x^4 + 13*x^2 - 15*x - 2

Complex:
d * e = 10*x^5 + (-20-17j)*x^4 + (26-18j)*x^3 + (-21-11j)*x^2 + (10+16j)*x + (11-17j)


In [18]:
# 4. деление
print('Real:')
print('a / b = {}'.format(a / b))
print('b / c = {}'.format(b / c))
print('\nComplex:')
print('d / e = {}'.format(d / e))

Real:
a / b = (x^3 - x^2 + 6.0*x - 10.0, 30.0*x - 19.0)
b / c = (0, x^2 + x - 2)

Complex:
d / e = (2.5*x^3 + (5+8.25j)*x^2 + (-9.125+28.5j)*x + (-90.5+34.1875j), (-531.9375-312.75j))


In [19]:
# 5. быстрое возведение в степень
print('Real:')
print('c^4 = {}'.format(c**4))
print('\nComplex:')
print('e^4 = {}'.format(e**4))

Real:
c^4 = 625*x^12 - 500*x^11 - 850*x^10 + 580*x^9 + 481*x^8 - 232*x^7 - 136*x^6 + 32*x^5 + 16*x^4

Complex:
e^4 = (16+0j)*x^4 + (-128-160j)*x^3 + (-216+960j)*x^2 + (1888-920j)*x + (-1519-720j)


In [20]:
# 7. дифференцирование
print('Real:')
print('dif({}) = {}'.format(c, c.differentiate()))
print('\nComplex:')
print('dif({}) = {}'.format(d, d.differentiate()))

Real:
dif(- 5*x^3 + x^2 + 2*x) = - 15*x^2 + 2*x + 2

Complex:
dif(5*x^4 + 4j*x^3 + (3-1j)*x^2 - 2*x + (1+3j)) = 20*x^3 + 12j*x^2 + (6-2j)*x - 2


In [21]:
# 8. интегрирование
print('Real:')
print('integral({}) = {}'.format(b, b.integrate(const=1)))
print('\nComplex:')
print('integral({}) = {}'.format(e, e.integrate(const=1)))

Real:
integral(x^2 + x - 2) = 0.333*x^3 + 0.5*x^2 - 2.0*x + 1

Complex:
integral(2*x + (-4-5j)) = x^2 + (-4-5j)*x + 1


In [22]:
# 9. вычисление значения в точке
print('Real:')
print('a({}) = {}'.format(-4, a(-4)))
print('\nComplex:')
print('e({}) = {}'.format(15, e(15)))

Real:
a(-4) = -1279

Complex:
e(15) = (26-5j)


### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [23]:
class CustomRange:
    def __init__(self, *args):
        if len(args) == 1:
            self.__check_int(*args)
            self.start, self.stop, self.step = 0, args[0], 1
        elif len(args) == 2:
            self.__check_int(*args)
            self.start, self.stop, self.step = args[0], args[1], 1
        elif len(args) == 3:
            self.__check_int(*args)
            self.start, self.stop, self.step = args[0], args[1], args[2]
            if self.step == 0:
                raise ValueError("CustomRange() arg 'step' must be not 0")
        else:
            raise TypeError("CustomRange() expected at most 3 arguments, got {}".format(len(args)))
        self.i = 0
    
    @staticmethod
    def __check_int(*args):
        for arg in args:
            if type(arg) != int:
                raise TypeError("{} object cannot be interpreted as an integer".format(type(arg)))
                
    def __next__(self):
        current = self.start + self.step*self.i
        if self.step > 0:
            continue_condition = self.i >= 0 and current < self.stop
        else:
            continue_condition = self.i >= 0 and current > self.stop
        while continue_condition:
            self.i += 1
            return current
        raise StopIteration
    
    def __iter__(self):
        return self
    
    def __repr__(self):
        if self.step == 1:
            custom_range = "CustomRange({}, {})".format(self.start, self.stop)
        else:
            custom_range = "CustomRange({}, {}, {})".format(self.start, self.stop, self.step)
        return custom_range

In [25]:
import random

for i in range(3):
    start, stop, step, sign = random.randint(-2, 3), random.randint(-8, 9), random.randint(1, 3), random.choice([-1,1])
    print("start={}, stop={}, step={}".format(start, stop, step*sign))
    print("{}: {}".format(CustomRange(start, stop, sign*step), list(CustomRange(start, stop, sign*step))))
    print("{}: {}".format(range(start, stop, sign*step), list(range(start, stop, sign*step))))
    print()

start=-2, stop=1, step=1
CustomRange(-2, 1): [-2, -1, 0]
range(-2, 1): [-2, -1, 0]

start=2, stop=-4, step=-1
CustomRange(2, -4, -1): [2, 1, 0, -1, -2, -3]
range(2, -4, -1): [2, 1, 0, -1, -2, -3]

start=2, stop=0, step=3
CustomRange(2, 0, 3): []
range(2, 0, 3): []



In [26]:
CustomRange(1,2,3,4,5)

TypeError: CustomRange() expected at most 3 arguments, got 5

In [27]:
CustomRange(1, 2, 0)

ValueError: CustomRange() arg 'step' must be not 0

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

In [28]:
class PrimaryKey:
    def __init__(self):
        self.existing_values = []
        
    def __set__(self, instance, value):
        if value is None:
            raise ValueError("Primary Key must be not null") 
        elif value in self.existing_values:
            raise ValueError("Primary Keys must be unique. Key '{}' already exists".format(value))
        self.existing_values.append(value)

    def __get__(self, instance, owner=None):
        return self.existing_values
    
class Table:
    primary_key = PrimaryKey()
    def __init__(self):
        self.data = {}
    
    def insert_row(self, key, row):
        self.primary_key = key
        self.data[key] = row
    
    def delete_row(self, key):
        for i in range(len(self.primary_key)):
            if self.primary_key[i] == key:
                break
        del self.primary_key[i]
        del self.data[key]
    
    def show_table(self):
        for key in self.primary_key:
            print("{}: {}".format(key, self.data[key]))

In [29]:
table_example = Table()

table_example.insert_row('а111аа90', ['bmw', 2016])
table_example.insert_row('к011фт177', ['audi', 2018])

In [30]:
table_example.show_table()

а111аа90: ['bmw', 2016]
к011фт177: ['audi', 2018]


In [31]:
table_example.insert_row('а111аа90', ['reno', 2018])

ValueError: Primary Keys must be unique. Key 'а111аа90' already exists

In [32]:
table_example.delete_row('а111аа90')

In [33]:
table_example.show_table()

к011фт177: ['audi', 2018]


In [34]:
table_example.insert_row('а111аа90', ['reno', 2018])

In [35]:
table_example.show_table()

к011фт177: ['audi', 2018]
а111аа90: ['reno', 2018]


### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [36]:
PSIF_MIN_VALUE = 0
PSIF_MAX_VALUE = 32767

class PositiveSmallIntegerField:
    def __init__(self, value=None):
        self.value = value
        
    def __set__(self, instance, value):
        if value < PSIF_MIN_VALUE or value > PSIF_MAX_VALUE:
            raise ValueError("PositiveSmallIntegerField takes values from {} to {}".format(PSIF_MIN_VALUE, PSIF_MAX_VALUE))
        self.value = value

    def __get__(self, instance, owner):
        return self.value


class PositiveSmallIntegerVariable:
    value = PositiveSmallIntegerField()
    def __init__(self, value):
        self.value = value

In [37]:
b = PositiveSmallIntegerVariable(-10)

ValueError: PositiveSmallIntegerField takes values from 0 to 32767

In [38]:
b = PositiveSmallIntegerVariable(100)

In [39]:
b.value = 1798031

ValueError: PositiveSmallIntegerField takes values from 0 to 32767

In [40]:
b.value

100

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [41]:
import time
import random

class Timer():
    def __init__(self):
        self.begin = time.time()
    
    def __enter__(self):
        return
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Spent time: {:.4f}".format(time.time() - self.begin))

In [43]:
with Timer():
    time.sleep(random.randint(1, 5)*random.random())

Spent time: 0.3851
